## Spatial Transcriptomics simulation with spatial dependencies

- **Fix library-size sampling noise** <br>
We sample single cell indices with similar library size to the same synthetic spots. <br><br>

- **Add spatial dependencies with 2D Gaussian Process simulations**<br>
There is smooth transition of both cell-type specific and overall spatial density patterns (e.g. similar density for adjacent spots).

In [ ]:
import os
import sys
import math
import json

import numpy as np
import pandas as pd
import seaborn as sns
import pymc3 as pm

import scanpy as sc
import torch
import gpytorch

import matplotlib.pyplot as plt

### Cell2Location simulation

**Step 1: simulate per cell abundance for each spatial pattern**
- $r$: Spatial patterns: (1). ubiquitous ($U = \{u_{1},
\dots,u_{n_1}\}$); (2). regional ($R = \{r_{1},
\dots,r_{n_2}\}$) (consisting of sub-patterns of number $n_1$, $n_2$)<br>
- $f$: cell type (in our case $f=9$<br>
- $a$: abundance (density): (1). high density (H); (2). low density (L)<br>
- $x_{r,f}$: binary assignment of cell types to spatial patterns
    - e.g. $x_{r=U, f=k} = 0$:  cell type $k$ exist in ubiquitous spatial pattern

- $d_f$: per cell type average abundance: 
    - $d_f^{r=U, a=L} \sim \text{Gamma}(5, 5)$
    - $d_f^{r=U, a=H} \sim \text{Gamma}(14, 5)$
    - $d_f^{r=R, a=L} \sim \text{Gamma}(5, 5)$
    - $d_f^{r=R, a=H} \sim \text{Gamma}(14, 5)$

- $s$: spatial coordinates (e.g. $s = (50, 50)$)<br>
- $z_{s,r}$: spatial abundance for spatial patterns $r$
    - $z_{s, r=U} \sim  \text{GP}(\mu=0, \eta=0.5, \text{bw}=bw_{r})$
    - $z_{s, r=R} \sim \text{GP}(\mu=0, \eta=1.5, \text{bw}=bw_{r})$<br>
    (high $\eta$ $\rightarrow$ high GP variance, $bw_{r} \sim \text{Gamma}(9.6, 1.2)$<br>
    $z_{s,r} = \text{softmax}(z_{s,r}) = \dfrac{ e^{z_{s,r}} }{ \sum e^{z_{s,r}}  }$  - ensure positive scale<br>
    $z_{s,r} = \dfrac{z_{s,r}}{\max_s z_{s,r}}$

- $w_{s,f}$: Per cell-type abundance for each location $s$<br>
    - $w_{s,f} = \sum_{r} (z_{s,r} x_{r,f}) \cdot q_{s,f}$ <br>
    $q_{s,f} \sim \text{LogNormal}(\mu=0, \sigma=0.35)$


Note:
exponentiated quadratic kernel: $k(x_1, x_2) = \sigma^2 \exp{- \dfrac{\Vert x_1 - x_2 \Vert^2}{2 l^2}}$
- $\sigma^2$: overall variance (amplitude)
- $l$: lengthscale

In [ ]:
# Code from cell2location simulations
# Reference: https://github.com/vitkl/cell2location_paper

def kernel(X1, X2, l=1.0, eta=1.0):
    """
    Isotropic squared exponential kernel. Computes 
    a covariance matrix from points in X1 and X2.
    
    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        Covariance matrix (m x n).
    """
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return eta**2 * np.exp(-0.5 / l**2 * sqdist)
    
def generate_grid(n=[50, 50]):
    n1, n2 = n 
    x1 = np.linspace(0, 100, n1)[:,None] #saptial dimensions 
    x2 = np.linspace(0, 100, n2)[:,None] #saptial dimensions 

    # make cartesian grid out of each dimension x1 and x2
    return pm.math.cartesian(x1[:,None], x2[:,None]), x1, x2

def random_GP(X, 
              x1=1, x2=1, #coordinates
              n_variables = 5, # zones
              eta_true = 5, #variance, defines overlapping
              l1_true=[8, 10, 15], #bw parameter
              l2_true=[8, 10, 15]
             ):

    #cov1, cov2 = kernel(x1, x1, l=l1_true), kernel(x2, x2, l=l2_true)
    K = [np.kron(kernel(x1, x1, l=l1_true[i], eta=eta_true), 
                 kernel(x2, x2, l=l2_true[i], eta=eta_true)) 
         for i in range(n_variables)]
    
    #samples from GP
    gaus_true = np.stack([
        np.random.multivariate_normal(np.zeros(X.shape[0]), 2*K[i]) 
        for i in range(n_variables)
    ]).T 
    
    N_true = (np.exp(gaus_true).T / np.exp(gaus_true).sum(axis=1)).T #softmax transform 
    return N_true


# TODO:
# Try increasing GP eta (variance) value --> more inhomogenous cell-type density 
def sample_GP(locations, 
              ct_labels,
              n_spec,
              n_ubiq,
              bw_spec,
              bw_ubiq,
              x1, 
              x2,
              eta_spec=2,
              eta_ubiq=1
             ):
    """
    Sample abundances with GP
    
        Higher `eta` values represent higher transition rates
        from high-density <--> low -density region
        
    """
    sparse_abundances = random_GP(
        X=locations, 
        x1=x1, 
        x2=x2, 
        n_variables=n_spec, 
        eta_true=eta_spec,
        l1_true=bw_spec,
        l2_true=bw_spec
    )
    
    sparse_abundances = sparse_abundances / sparse_abundances.max(0)
    sparse_abundances[sparse_abundances < 0.1] = 0

    uniform_abundances = random_GP(
        X=locations, 
        x1=x1, 
        x2=x2, 
        n_variables=n_ubiq, 
        eta_true=eta_ubiq,
        l1_true=bw_ubiq,
        l2_true=bw_ubiq
    )
    
    uniform_abundances = uniform_abundances / uniform_abundances.max(0)
    uniform_abundances[uniform_abundances < 0.1] = 0

    abundances = np.concatenate([sparse_abundances, uniform_abundances], axis=1)
    
    return pd.DataFrame(
        abundances, 
        index=[f'location_{i}' for i in range(abundances.shape[0])],
        columns=ct_labels
    )


def plot_spatial(values, n=[50,50], nrows=5, names=['cell type'],
                 vmin=0, vmax=1):
    
    n_cell_types = values.shape[1]
    n1, n2 = n 
    ncols = np.ceil((n_cell_types+1) / nrows).astype(np.uint16)
    for ct in range(n_cell_types):
        try:
            plt.subplot(nrows, ncols, ct+1)
            plt.imshow(values[:,ct].reshape(n1,n2).T, 
                       cmap=plt.cm.get_cmap('magma'),
                       vmin=vmin, vmax=vmax
                      )
            plt.colorbar()
            if len(names) > 1:
                plt.title(names[ct])
            else:
                plt.title(f'{names[0]} {ct+1}')
                
        except ValueError:
            continue

    try:
        plt.subplot(nrows, ncols, n_cell_types+1) 
        plt.imshow(values.sum(axis=1).reshape(n1,n2).T, 
                   cmap=plt.cm.get_cmap('Greys'))
        plt.colorbar()
        plt.title('total')
    
    except ValueError as e:
        print(e)


**Modification:** <br>
Direct assignment of ubiquitous & specific cell types:
- Ubiquitous: B-cell, T-cell, Endothelial, Plasmablast
- Specific: Normal_Epithelial, Cancer_Epithelial, CAFs, PVL, Myeloid

(Note: unify cell-type names: Cancer_Epithelial = TNBC)

In [ ]:
def get_ct_name(ct_map, idxs):
    try:
        return [ct_map[i] for i in idxs]
    except KeyError(e):
        print('Index {} not included in cell type map'.format(i))
        return -1

In [ ]:
# Define cell types & cell-type assignment to spatial patterns
ct_names = [
    'B-cells',
    'CAFs',
    'Endothelial',
    'Normal_Epithelial',
    'Myeloid',
    'PVL',
    'Plasmablasts',
    'T-cells',
    'Cancer_Epithelial'
]

ubiq_ct_names = [
    'B-cells',
    'Endothelial',
    'T-cells',
    'Plasmablasts'
]

spec_ct_names = [
    'CAFs',
    'Normal_Epithelial',
    'Myeloid',
    'PVL',
    'Cancer_Epithelial'
]

ct_maps = {i: ct for (i, ct) in enumerate(ct_names)}

n_cts = len(ct_names) # number of cell types
n_spec_cts = len(spec_ct_names)
n_ubiq_cts = len(ubiq_ct_names)

spec_cell_types = [
    idx
    for idx, name in enumerate(ct_names)
    if name in set(spec_ct_names)
]

ubiq_cell_types = [
    idx
    for idx, name in enumerate(ct_names)
    if name in set(ubiq_ct_names)
]

# numerical representation of cell types (NOTE: order super important!)
cell_types = np.concatenate([spec_cell_types, ubiq_cell_types])

**TODO**

Simulate whole canvas and mask out tissue regions with true spatial regions (simualate hotspot of specific cell types (using localized patterns).


Load image & spot coordinates:

In [ ]:
from skimage import io
import json

In [ ]:
# Please re-specify the spatial data path
# Data: CID44971 
spatial_path = '../../data/wu_data/CID44971/st/'
sample_id = 'CID44971'


# Load real ST data (CID44971)
adata = sc.read_h5ad(
    os.path.join(spatial_path, sample_id+'.h5ad')
)

adata.var_names_make_unique()

# Load image & locations
"""
with open('../../data/1a/spatial/scalefactors_json.json', 'r') as f:
    scale_info = json.load(f)
    scale = scale_info['tissue_hires_scalef']
    
locs = np.round(adata.obsm['spatial'] * scale).astype(int)
size = hist_img.shape
"""
print()

In [ ]:
def _random_assign(n):
    """Random one-to-one assignemnt by permuting an NxN identity matrix"""
    mat = np.random.permutation(np.diag(np.ones(n)))
    return mat

def assign_ct_pattern(ubiq_cts, 
                      spec_cts, 
                      n_spec_patterns,
                      n_ubiq_patterns, 
                     ):
    """
    Assign cell-type categories to spatial pattern categories
    """
    n_ubiqs = len(ubiq_cts) # number of ubiquitous cell types
    n_specs = len(spec_cts) # number of specific cell types
    
    # Modification:
    # For both ubiquitous & spatial pattern, only allow one-to-one mapping
    spec_assignment = _random_assign(n_spec_patterns)
    ubiq_assignment = _random_assign(n_ubiq_patterns)
    
    """
    ubiq_assignment = np.random.binomial(
        n=1,
        p=0.4,
        size=(len(ubiq_cts), n_ubiq_patterns)
    )
    """
    
    mat_spec = np.concatenate(
        (spec_assignment, 
         np.zeros((len(spec_cts), n_ubiq_patterns))
        ),
        axis=1
    )

    mat_ubiq = np.concatenate(
        (np.zeros((len(ubiq_cts), n_spec_patterns)),
         ubiq_assignment
        ),
        axis=1
    )
    
    idx_names = np.concatenate((spec_cts, ubiq_cts))
    col_names = ['Spec_'+str(i+1) for i in range(n_spec_patterns)] + \
                ['Ubiq_'+str(i+1) for i in range(n_ubiq_patterns)]
    
    
    patterns_df = pd.DataFrame(
        np.concatenate(
            (mat_spec, mat_ubiq),
            axis=0
        ).astype(np.uint8),
        index=idx_names,
        columns=col_names
    )
    
    patterns_df.index.name='cell_types'
    
    return patterns_df

Increase GP bandwidth:

In [ ]:
# Generate matrix of which cell types are in which zones
# Specify number of distinct spatial patterns under each category
"""
    Assignment of cell-type to specific spatial patterns 
    (either `Spec` - specific or `Ubiq` - Ubiquitous)
"""


np.random.seed(1000)
n_experiments = 1
n_locations = [50, 50]

ct_patterns_df = assign_ct_pattern(
    ubiq_cell_types,
    spec_cell_types,
    n_spec_patterns=n_spec_cts,
    n_ubiq_patterns=n_ubiq_cts
)

# Sample bandwidth parameter
mean_var_ratio = 1.5
mean_spec = 12
mean_ubiq = 8

bandwidth_spec = np.random.gamma(
    mean_spec * mean_var_ratio, 1 / mean_var_ratio,
    size=n_spec_cts
)

bandwidth_ubiq = np.random.gamma(
    mean_ubiq * mean_var_ratio, 1 / mean_var_ratio,
    size=n_ubiq_cts
)

locations_1, x1, x2 = generate_grid(n=n_locations)
locations = np.concatenate([locations_1 for _ in range(n_experiments)], axis=0)


# Sample per-cell-type, per-spot abundance
abundances_df = pd.DataFrame()
for e in range(n_experiments):
    
    abundances_df_1 = sample_GP(
        locations=locations_1, 
        ct_labels=cell_types,
        n_spec=n_spec_cts,
        n_ubiq=n_ubiq_cts,
        bw_spec=bandwidth_spec,
        bw_ubiq=bandwidth_ubiq,
        x1=x1, x2=x2
    )
    
    abundances_df_1.index = [f'exper{e}_{l}' for l in abundances_df_1.index]
    abundances_df = pd.concat((abundances_df, abundances_df_1), axis=0)
    
    plt.figure(figsize=(2*5+5,3*5+5))
    plot_spatial(abundances_df_1.values, n=n_locations, nrows=5, names=np.arange(n_cts+1))
    plt.tight_layout()
    plt.show()

Generate per-spot spatial pattern:<br>
For each cell type:
- matrix dot spatial pattern `abundance_df` with binary assignment of spatial patterns `ct_patterns_df`
<br><br>
    Variable notation:

    - `abundance_df` ($\mathbb{R}^{s \times f}$): spatial abundance for each pattern
    - `ct_patterns_df`: binary assignment of which cell type belongs to which spatial pattern
    - `cell_abundances_df` ($\mathbb{R}^{s \times f}$): spatial abundance for each cell type & spot

Latest modification: <br>
For each cell type, we simulate independent $q_{s,c,f}$, the multiplier to generate cell abundance from spatial pattern assigment (0, 1)

In [ ]:
np.random.seed(1000)

cell_abundances = np.dot(abundances_df, ct_patterns_df.T)


q_sf = np.tile(
    np.random.lognormal(0.5, 0.5, size=n_cts),
    reps=(n_locations[0] * n_locations[1], 1),
)

cell_abundances = cell_abundances * q_sf
cell_abundances_df = pd.DataFrame(
    cell_abundances, 
    index=abundances_df.index,
    columns=ct_patterns_df.index
)

cell_abundances_df.columns = spec_ct_names + ubiq_ct_names

plt.figure(figsize=(3*5+5, 3*10+5))
plot_spatial(cell_abundances_df.values, 
             n=n_locations, nrows=5, 
             names=cell_abundances_df.columns, vmax=None)
plt.tight_layout()
plt.show()

Discretize spot-wise cell count from cell_abundances by taking `ceiling`<br>
Generate ground-truth proportions:

In [ ]:
cell_count_df = np.ceil(cell_abundances_df).astype(np.uint16)
proportion_df = cell_count_df / np.expand_dims(cell_count_df.sum(1), 1)

In [ ]:
cell_count_df.sum(1).value_counts()

In [ ]:
proportion_df.head()

**Step 2** 

Sample cell indices from scRNA-seq for each spot, construct synthetic `(S x G)` counts by sampling cells with similar library size.

(1). Load scRNA-seq matrix counts & metadata (cell-type annotation)

In [ ]:
sc_path = '../../sc_transcriptome_ana/data/wu_data/CID44971/scrna/'

# scRNA-seq count matrix
df_sc = pd.read_csv(
    os.path.join(sc_path, 'bc_sc.csv'),
    index_col=[0],
    header=[0]
)

# scRNA-seq metadata
df_meta = pd.read_csv(
    os.path.join(sc_path, 'metadata.csv'), index_col=[0]
)

annots = df_meta['celltype_major']
df_sc.index.name = 'Barcode'
df_sc['cell_type'] = annots.to_list()
df_sc.set_index('cell_type', inplace=True, append=True)


In [ ]:
annots.value_counts()

In [ ]:
df_sc.head()

(2). Fit raw library size with $\text{LogNormal}$ distribution:

**Fix inhomogeneous library size for spots captured by same synthetic spots**

- (i). Fit the library size of provided ST data with `LogNormal` r.v.
- (ii). Sample each synthetic spot $s$ with a library size $l_s$ draw from `LogNormal` r.v. retrieved from step (1)
- (iii). For each single-cell assignment to the given synthetic spot $s$, sample the library size with poisson distribution: $l_{s_{i}} \sim \text{Poisson}(l_s)$
    - For now, we don't sample per-cell library size for each spot with Poisson r.v., fix the expected library size for all scRNA-seq indices in spot $s$ as $l_s$ 
- (iv). For each single-cell assignment, and it's library

In [ ]:
from scipy.stats import lognorm, poisson, gamma

In [ ]:
# Fit library size r.v. with scRNA-seq data
# plot the original histogram with the simulated one from fitted parameters

libsize = df_sc.values.sum(1)
s, loc, scale = lognorm.fit(libsize)

rvs = lognorm.rvs(s, loc=loc, scale=scale, size=1000)
plt.figure(figsize=(10, 6))

plt.hist(libsize, bins=100, density=True, label='Raw libsize',
         color='blue', edgecolor='white', alpha=0.5)
plt.hist(rvs, bins=100, density=True, label='Fitted libsize',
         color='green', edgecolor='white', alpha=0.5)

x = np.linspace(libsize.min(), libsize.max(), len(libsize)+1)
pdf = lognorm.pdf(x, s, loc, scale)
plt.plot(x, pdf, '--', color='r')
plt.legend()

plt.show()

(3). Generate spot x gene matrix by synthesizing `pseudo-spots` with cell-type specific density & library-size 

Some visualization (archived):

In [ ]:
# Visualize cell count per cell-type 
plt.figure(figsize=(3*5+5, 3*10+5))

plot_spatial(cell_count_df.values, 
             n=n_locations, nrows=4, 
             names=cell_abundances_df.columns, vmax=None)

In [ ]:
# Visualize per cell-type capture rate
eps = 1e-10

cell_capture_rate_df = cell_abundances_df / (cell_count_df + eps)

plt.figure(figsize=(3*5+5, 3*10+5))
plot_spatial(cell_capture_rate_df.values,
             n=n_locations, nrows=4,
             names=cell_abundances_df.columns, vmax=None)

plt.suptitle('Capture rate per cell type', y=0.9, fontsize=20)
plt.show()

In [ ]:
# Visualize avg. cell count per spot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 5))

sns.heatmap(cell_count_df[cell_count_df>0].mean(1).values.reshape(50, -1), ax=ax1)
ax1.set_title('Avg. non-empty cell count per spot')
ax1.axis('off')

sns.heatmap(cell_count_df[cell_count_df>0].var(1).values.reshape(50, -1), ax=ax2)
ax2.set_title('Variance of non-empty cell count per spot')
ax2.axis('off')

plt.tight_layout()
plt.show()

Simulate library size for each spot:

First, Check overall capture ratio from scRNA-seq to visium (CID44971 scRNA - CID44971 ST) <br>

We scale the true capture ratio from scRNA-seq to ST as follows:<br>
Median library size ratio ($r = \dfrac{\text{ST library size}}{\text{scRNA-seq library size}}$) divided by average `# cells` captured within each synthetic spot:<br>

$$r_{scaled} = \dfrac{r}{\text{avg. # cells captured}}$$

In [ ]:
libsize = df_sc.values.sum(1)
libsize_st = adata.X.A.sum(1)

cap_ratio = np.median(np.log1p(libsize_st)) / np.median(np.log1p(libsize))
cap_ratio_per_spot = cap_ratio / cell_count_df.sum(1).mean()
cap_ratio_per_spot

In [ ]:
def find_nonzero_cts(df):
    """
    Find expressed (non-zero) cell types per spot
    """
    cols = df.columns
    return (df > 0).apply(
        lambda x: list(cols[x.values]), axis=1
    )


def find_sc_index(df, cell_type, libsize_raw, l, n_nbr=3):
    """
    Find the scRNA-seq cell with close library size to given expected library size of 
    the synthetic spot (s). 
    
    Either randomly sample the index out of n closest libsize or 
    deterministically choose the cell index with the closest libsize
    
    Returns
    -------
    expr : list
        Gene expression of the selected scRNA-seq cell
    """
    
    # Bug: the index in subsetted libsize different from the original count index!!!
    cell_types = np.unique(df.index.get_level_values(1))
    assert cell_type in cell_types, 'Unexpected cell type {}'.format(cell_type)
    
    # 'Mask out' indices other than the given cell type
    mask = df.index.get_level_values(1) == cell_type
    libsize = libsize_raw.copy()
    libsize[~mask] = -np.inf   
    
    n = np.random.randint(n_nbr) if n_nbr > 0 else 0
    idx = np.abs(libsize - l).argsort()[n]
    expr = df.iloc[idx].to_numpy()
    
    return expr


def fit_st_libsize(libsize, idxs):
    """
    Fit library size from scRNA-seq data & sample expected library size 
    to each synthetic Spatial Transcriptomics spot
    """
    
    s, loc, scale = lognorm.fit(libsize)
    l = pd.Series(
        lognorm.rvs(s, loc=loc, scale=scale, size=len(idxs)),
        index=idxs
    )
    
    return np.round(l).astype(np.uint32)
    

def simulate_spots(df, count_df, expr_factors, l, ratio=0.16, verbose=False):
    """
    Simulate spot x gene synthetic matrix given 
    (1). per-cell density; (2). library size for each spot
    """
    n_spot = count_df.shape[0]
    n_genes = df.shape[1]
    libsize = df.sum(1).to_numpy().astype(np.float64)
    
    sims = np.zeros((n_spot, n_genes))
    
    for i, cell_types in enumerate(expr_factors):
        if verbose and i % 100 == 0:
            print('Simulating spot {}...'.format(i))
            
        l_s = l[i] # expected library size for given spot
        spot_expr = np.zeros(n_genes)
        
        cell_count = count_df[cell_types].iloc[i]
        n_cells = cell_count.sum().astype(np.uint32)
        
        # Sample expected library size to each single-cell index 
        # captured to current spot
        # for now fix the same expected library size for all scRNA-seq to the given spot
        """
        expected_libsize = poisson.rvs(
            l[i], 
            size=n_cells
        )
        """
        
        cell_type_freqs = np.hstack([
            np.repeat(cell_type, count)
            for (cell_type, count) in zip(cell_types, cell_count)
        ])
        
        # for count, cell_type in zip(expected_libsize, cell_type_freqs):
        for cell_type in cell_type_freqs:
            ct_expr = find_sc_index(df, cell_type, libsize, l_s, n_nbr=0)
            spot_expr += ct_expr
            
        spot_expr = np.round(spot_expr * ratio).astype(np.uint64)
        sims[i] = spot_expr
        
    sims_df = pd.DataFrame(
        sims,
        index=count_df.index,
        columns=df.columns
    )
    
    return sims_df
        

- Sample library size $l_s$ for each synthetic spot from the r.v. fitted from scRNA-seq.<br>
  ($l_s$ restricts the max. number of gene expression count for each spot $s$)<br>
- Simulate synthetic expression ($X_{s,g}$):

In [ ]:
# Get list of expressed (non-zero) cell type for each spot
expressed_factors = find_nonzero_cts(cell_count_df)

# Fit library size to each synthetic spot
l_s = fit_st_libsize(
    libsize=df_sc.values.sum(1), 
    idxs=cell_abundances_df.index
)

synth_df = simulate_spots(
    df_sc,
    cell_count_df,
    expressed_factors,
    l_s,
    ratio=cap_ratio_per_spot,
    verbose=True
)

In [ ]:
synth_df.head()

Display library size per spot:

In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(synth_df.sum(1).to_numpy().reshape(50, -1))
plt.axis('off')
plt.title('Library size per spot')
plt.show()

In [ ]:
# Comparison of real & synthetic ST library size histogram

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

ax1.hist(adata.X.sum(1), bins=60, density=True, edgecolor='white')
ax1.set_title('Library size - real ST matrix')

ax2.hist(synth_df.sum(1), bins=60, density=True, edgecolor='white')
ax2.set_title('Library size - synthetic ST matrix')

plt.show()

Save simulations to output
- Spot x Gene synthetic matrix
- Ground-truth cell count (density)
- Ground-truth cell proportion

In [ ]:
%ls ../../sc_transcriptome_ana/simulation/data_cell2loc/

In [ ]:
# Save simulations to file
out_path = '../../sc_transcriptome_ana/simulation/data_cell2loc/'

In [ ]:
synth_df.to_csv(
    os.path.join(out_path, 'counts.st_synth.csv'),
    index=True
)

cell_count_df.to_csv(
    os.path.join(out_path, 'members.st_synth.csv'),
    index=True
)

proportion_df.to_csv(
    os.path.join(out_path, 'proportions.st_synth.csv'),
    index=True
)

---

#### [Benchmark] Check correlation between signature mean gexp. & ground-truth proportion

In [ ]:
# Load signature gene dict

# specify signature genes directory
sig_genes_df = pd.read_csv('../../sc_transcriptome_ana/simulation/data/tnbc_signature.csv', header=[0])
sig_genes_dict = sig_genes_df.to_dict(orient='list')

In [ ]:
out_path = '../../sc_transcriptome_ana/simulation/data_cell2loc/'

synth_df = pd.read_csv(
    os.path.join(out_path, 'counts.st_synth.csv'),
    index_col=[0],
    header=[0]
)

cell_count_df = pd.read_csv(
    os.path.join(out_path, 'members.st_synth.csv'),
    index_col=[0],
    header=[0]
)

proportion_df = pd.read_csv(
    os.path.join(out_path, 'proportions.st_synth.csv'),
    index_col=[0],
    header=[0]
)


In [ ]:
# Parallel comparison with Stereoscope-like simulation (without normalization)

stereo_sim_path = '../../sc_transcriptome_ana/simulation/data_stereo/'

stereo_synth_df = pd.read_csv(
    os.path.join(stereo_sim_path, 'counts.st_synth.csv'),
    index_col=[0],
    header=[0]
)

stereo_cell_count_df = pd.read_csv(
    os.path.join(stereo_sim_path, 'members.st_synth.csv'),
    index_col=[0],
    header=[0]
)

stereo_proportion_df = pd.read_csv(
    os.path.join(stereo_sim_path, 'proportions.st_synth.csv'),
    index_col=[0],
    header=[0]
)

In [ ]:
def disp_density_gexp(expr_df, prop_df, sig_dict, normalize=False):
    """
    Display relation of ground-truth cell-type proportion vs. 
    average of cell-type specific marker gene expression 
    """
    
    cell_types = prop_df.columns
    assert len(sig_dict) == len(cell_types) and \
           np.array_equal(sorted(sig_dict.keys()), sorted(cell_types)), \
        "Unequal length of cell types in signature dictionary & ground-truth proportion dataframe"
    
    expr_df_copy = expr_df.copy()
    
    if normalize:
        med_libsize = expr_df_copy.sum(1).median()
        expr_df_copy = expr_df_copy.div(expr_df_copy.sum(1), axis=0) * med_libsize
    
    # Average expression of cell-type specific marker genes
    sig_expr = np.zeros(prop_df.shape) 
    for i, ct in enumerate(prop_df.columns):
        sig_genes = np.intersect1d(sig_dict[ct], expr_df.columns)
        expr = expr_df_copy[sig_genes].mean(1)
        sig_expr[:, i] = expr
        
    sig_expr = sig_expr / np.expand_dims(sig_expr.sum(1), axis=1) # normalize expression to proportion
    sig_expr_df = pd.DataFrame(
        sig_expr,
        index=expr_df.index,
        columns = prop_df.columns
    )
    
    # Plotting
    r = 0
    c = 0    
    nrow = np.ceil(len(cell_types) // 3).astype(np.uint8)
    fig, axes = plt.subplots(nrow, 3, figsize=(15, 15))
    
    for i, ct in enumerate(cell_types):
        if c % 3 == 0 and c > 0:
            r += 1
            c = 0
        
        xx = prop_df[ct].to_numpy()
        yy = sig_expr_df[ct].to_numpy()
        
        ax = axes[r, c]
        ax.scatter(xx, yy, alpha=0.5, marker='.', cmap=plt.cm.Greens)
        ax.set_xlim([0, 1])
        ax.set_ylim([0, 1])
        ax.set_xlabel('Ground-truth proportion')
        ax.set_ylabel('Mean sig gexp')
        ax.set_title(ct)
        
        c += 1
        
    plt.tight_layout()
    # plt.suptitle('Cell2Location simulation (w/ spatial dependencies)', y=1.05, fontsize=20)
    plt.show()
    

In [ ]:
# Cell2location simulation (w/ spatial dependencies)
disp_density_gexp(synth_df, proportion_df, sig_genes_dict, normalize=False)

In [ ]:
# Stereoscope simulation (without spatial dependencies)
disp_density_gexp(stereo_synth_df, stereo_proportion_df, sig_genes_dict, normalize=False)

---